In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
import sys
import os
import time
import scipy
import pickle
from importlib import reload
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from sklearn.metrics import r2_score
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error
from collections import namedtuple
import keras_tuner as kt 
codebase_path = '/data/home/wpw035/Codebase'
sys.path.insert(0, codebase_path) #add path to my codebase models
from scipy.optimize import curve_fit

In [2]:
from DRP_utils import data_preprocessing as dp_nb
reload(dp_nb)
from DRP_utils import model_selection as ms_nb
reload(ms_nb)
from DRP_utils import testing as t_nb
reload(t_nb)
import Data_imports as di_nb
reload(di_nb)
import pairs_train_test_split as tts_nb
import Learning_curve as lc_nb

In [3]:
#input data
prot, rna, phospho_ls, one_hot_cls, one_hot_drugs, ic50_df1 = di_nb.read_input_data()
_all_cls = prot.index
_all_drugs = ic50_df1.columns
assert prot.shape[0] == rna.shape[0] == phospho_ls.shape[0]
assert phospho_ls.shape[0]  == one_hot_cls.shape[0]
prot.shape, rna.shape, phospho_ls.shape, one_hot_cls.shape, one_hot_drugs.shape

/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Number of missing prot values 0.386335609896865
num non overlapping prot and target cls: 10
num non overlapping cls: 930


((38, 8457), (38, 17417), (38, 22804), (38, 38), (345, 345))

## Featrue selection (FS) and data createing for each drug

In [4]:
#read in landmark genes for fs and find landmarks that overlap with rna data
landmark_genes = pd.read_csv(
    f'{codebase_path}/downloaded_data_small/landmark_genes_LINCS.txt',sep='\t')
landmark_genes.index = landmark_genes['Symbol']

overlapping_landmarks, _ = dp_nb.keep_overlapping(
    pd.DataFrame(landmark_genes['Symbol']), rna.T)

overlapping_landmarks = overlapping_landmarks['Symbol'].values

#create input data for each drug
x_all, x_drug, y_list = dp_nb.create_all_drugs(
    rna[overlapping_landmarks], one_hot_drugs, ic50_df1, _all_cls)

x_all = x_all.astype(np.float32)
x_drug = x_drug.astype(np.float16)

#fmt index to include drug cell line paris
cls_drugs_index = x_all.index + '::' + x_drug.index
x_all.index = cls_drugs_index
x_drug.index = cls_drugs_index
y_list.index = cls_drugs_index

x_all.shape, x_drug.shape, len(y_list)

((11583, 908), (11583, 345), 11583)

In [5]:
#use the same landmark genes, that were used for fs for rna datan
#for fs with prot data

#find overlapping landmark genes and prot features
overlapping_landmarks, _ = dp_nb.keep_overlapping(
    pd.DataFrame(landmark_genes['Symbol']), prot.T)

overlapping_landmarks = overlapping_landmarks['Symbol'].values

#create prot data for all drugs
x_all_prot, x_drug, y_list = dp_nb.create_all_drugs(
    prot[overlapping_landmarks], one_hot_drugs, ic50_df1, _all_cls)

#fmt index to include drug cell line paris
cls_drugs_index = x_all_prot.index + '::' + x_drug.index 
x_all_prot.index = cls_drugs_index
y_list.index = cls_drugs_index
x_drug.index = cls_drugs_index

x_all_prot = x_all_prot.astype(np.float32)
x_all_prot.shape

(11583, 721)

In [6]:
#one hot data creation for all drugs
x_hot, x_drug_hot, y_hot = dp_nb.create_all_drugs(
    one_hot_cls, one_hot_drugs, ic50_df1, _all_cls)

cls_drugs_index_hot = x_hot.index + '::' + x_drug_hot.index 

x_hot.index = cls_drugs_index_hot
x_hot.columns = np.arange(len(x_drug.columns), len(x_drug.columns) + len(x_hot.columns))
x_hot.shape

(11583, 38)

## Phospo FS

In [14]:
landmark_genes = pd.read_csv(
    f'{codebase_path}/downloaded_data_small/landmark_genes_LINCS.txt',sep='\t')
landmark_genes.index = landmark_genes['Symbol']

phos_genes = np.array([c.split('(')[0] for c in phospho_ls.columns])
unique_phos = pd.DataFrame(np.unique(phos_genes))
unique_phos.index = np.unique(phos_genes)
overlapping_landmarks, _ = dp_nb.keep_overlapping(
    pd.DataFrame(landmark_genes['Symbol']), unique_phos)

#read in targets of proteins data
dd_path = '/data/home/wpw035/down_data_small'
enz_sub = pd.read_csv(f'{dd_path}/enzsub_relations_phosSite_signor.csv', index_col=0)
enz_sub.index = enz_sub['enzyme_genesymbol']

#landmark genes with targets
lands_with_target = [g for g in landmark_genes.index if g in enz_sub.index]
land_targets = set(enz_sub.loc[lands_with_target]['substrate_genesymbol'])

overlapping_land_targets, _ = dp_nb.keep_overlapping(
    pd.DataFrame(land_targets, index=land_targets), unique_phos)

targets_also_landmarks = set(overlapping_land_targets.index).intersection(
    landmark_genes.index) 
print(len(targets_also_landmarks))

overlap_inds = []
for gene in targets_also_landmarks:
    overlap_inds.extend(np.where(phos_genes == gene)[0])
print(len(overlap_inds))

kep_cols = phospho_ls.columns[overlap_inds]
x_all_phos, x_drug, y_list = dp_nb.create_all_drugs(
    phospho_ls[kep_cols], one_hot_drugs, ic50_df1, _all_cls)

#fmt index to include drug cell line paris
cls_drugs_index = x_all_phos.index + '::' + x_drug.index 
x_all_phos.index = cls_drugs_index
y_list.index = cls_drugs_index
x_drug.index = cls_drugs_index

186
1064


In [15]:
x_all_phos

col.name,BCL2(T74);,LBR(T58);,LBR(S99);,LBR(S101);,LBR(S86);,LBR(S84);,DNM1L(S529);,DNM1L(M613);DNM1L(S616);,DNM1L(S616);,DNM1L(S607);,...,PLA2G4A(S458);,PLA2G4A(S434);PLA2G4A(S435);,PLA2G4A(S434);PLA2G4A(S435);PLA2G4A(S437);,PLA2G4A(S431);,PLA2G4A(T268);,TLK2(T98);,TLK2(S94);,TLK2(S761);,TLK2(S110);,TLK2(S110);TLK2(S115);
COLO-680N::(5Z)-7-Oxozeaenol,-1.287052,0.346725,-0.024961,-1.038458,-0.724768,-0.724768,-1.038458,1.471941,1.471941,-1.287052,...,-1.287052,-1.130207,-1.287052,0.748179,-1.287052,0.414010,0.558582,0.608332,0.998911,-1.287052
GDM-1::(5Z)-7-Oxozeaenol,-1.429010,-0.199680,0.503917,0.631856,-0.714868,-0.714868,-0.020987,1.350565,1.350565,0.210411,...,0.136731,0.877505,0.752149,0.967197,-1.429010,0.408668,0.627804,0.568997,1.046464,0.499662
HEL::(5Z)-7-Oxozeaenol,-1.613351,0.572971,0.351613,-0.480599,-1.015662,-1.015662,0.189010,1.510892,1.510892,0.631137,...,0.327475,-0.141741,-1.613351,0.681463,0.466131,0.419555,0.635565,0.470250,1.065829,0.102895
HL-60::(5Z)-7-Oxozeaenol,0.413451,0.747092,1.036664,0.660357,0.232528,0.232528,0.211431,1.355492,1.355492,-1.428863,...,0.601375,0.437609,0.271233,0.626967,0.283550,0.540028,0.798239,0.564024,0.923707,0.592933
JHH-2::(5Z)-7-Oxozeaenol,-1.224318,0.447709,0.147121,-0.990751,0.184555,0.184555,-0.990751,1.349678,1.349678,-1.224318,...,-1.224318,-1.076954,-1.224318,-0.011539,-1.224318,0.011151,-1.224318,0.074906,0.916161,-1.224318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SNU-387::rTRAIL,-1.215612,0.105196,0.262097,0.341835,-0.967579,-0.967579,0.280019,1.557620,1.557620,-1.215612,...,-1.215612,-1.059121,-1.215612,0.359945,-1.215612,0.357556,0.553098,0.507961,0.900685,-1.215612
SNU-398::rTRAIL,-0.988966,-0.831457,-0.831457,-0.739319,-0.739319,-0.739319,-0.739319,1.620929,1.620929,-0.988966,...,-0.988966,-0.831457,-0.988966,-0.988966,-0.988966,0.669203,0.612447,-0.988966,-0.739319,-0.988966
SNU-423::rTRAIL,-1.049877,0.814532,-0.891829,-0.799377,-0.363736,-0.363736,1.699472,1.688131,1.688131,-1.049877,...,-1.049877,-0.006160,-1.049877,0.515398,0.625330,0.229471,0.705072,-1.049877,0.950254,-1.049877
SNU-449::rTRAIL,-1.303739,-1.158293,-0.711848,-0.352822,-0.668073,-0.668073,-1.073213,1.559334,1.559334,-1.303739,...,-1.303739,-1.158293,-1.303739,-0.009883,-1.303739,0.478996,0.639933,0.561808,1.000157,-0.060066


## Model building

In [16]:
_input_shape=None
def build_cnn_kt(hp):
    if _input_shape == None:
        raise Exception('add input shape dim')
    phos_input = layers.Input(shape=(_input_shape, 1))
    x = layers.Conv1D(
        filters=hp.Int('filts', 8, 32, 8), kernel_size=16, 
        activation='relu')(phos_input)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(
        filters=hp.Int('filts', 8, 32, 8), kernel_size=8, activation='relu')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(hp.Int('units', 32, 258, 32), activation='relu')(x)
    x = layers.Dense(hp.Int('units', 32, 258, 32), activation='relu')(x)
    drug_input = layers.Input(shape = (xdrug_train.shape[1]))
    concatenated = layers.concatenate([x, drug_input])
    hidd = layers.Dense(hp.Int('units_hid', 32, 258, 32), activation='relu')(concatenated)
    hidd = layers.Dense(hp.Int('units_hid', 32, 258, 32), activation='relu')(hidd)
    output_tensor = layers.Dense(1)(hidd)
    model = tf.keras.Model([phos_input,drug_input], output_tensor)
    opt = tf.keras.optimizers.RMSprop(learning_rate=hp.Choice('lr', [1e-4, 1e-3]))
    model.compile(optimizer=opt, loss=tf.keras.metrics.mean_squared_error, metrics=['mae'])
    return model

In [17]:
_train_size  = 0.6 #train size relative to total data set size
lg_space = np.logspace(1, np.log2(64), base=2.0, num=5).astype(int)
lg2 = np.logspace(np.log2(64), np.log2(len(x_all) * _train_size),  base=2.0, num=50).astype(int)
lg_space = np.concatenate((lg_space, lg2))
lg_space = np.unique(lg_space)
lg_space

array([   2,    4,   11,   26,   64,   70,   77,   85,   93,  103,  113,
        125,  137,  151,  166,  183,  201,  221,  244,  268,  295,  325,
        358,  394,  433,  477,  525,  577,  635,  699,  769,  847,  932,
       1025, 1128, 1242, 1366, 1503, 1654, 1821, 2003, 2205, 2426, 2669,
       2937, 3232, 3557, 3914, 4307, 4740, 5215, 5739, 6315, 6949])

In [18]:
lg_space[0: 1]

array([2])

In [23]:
x_train_phos

col.name,BCL2(T74);,LBR(T58);,LBR(S99);,LBR(S101);,LBR(S86);,LBR(S84);,DNM1L(S529);,DNM1L(M613);DNM1L(S616);,DNM1L(S616);,DNM1L(S607);,...,PLA2G4A(S458);,PLA2G4A(S434);PLA2G4A(S435);,PLA2G4A(S434);PLA2G4A(S435);PLA2G4A(S437);,PLA2G4A(S431);,PLA2G4A(T268);,TLK2(T98);,TLK2(S94);,TLK2(S761);,TLK2(S110);,TLK2(S110);TLK2(S115);
KYSE-520::THZ-2-49,-1.188813,-1.025986,-1.025986,-0.930739,-0.930739,-0.930739,-0.930739,1.806736,1.806736,-1.188813,...,-1.188813,-1.025986,-1.188813,0.300292,0.452903,0.184968,0.816625,0.042717,0.945186,-1.188813
MOLM-13::Cabozantinib,0.628184,0.780134,0.909293,1.116339,1.123856,1.129601,0.355823,1.165814,1.165814,-1.511135,...,0.504763,0.811131,0.673139,0.822897,-1.511135,0.565244,0.871695,0.254203,0.949936,0.360083
SNU-398::PLK_6522,-0.988966,-0.831457,-0.831457,-0.739319,-0.739319,-0.739319,-0.739319,1.620929,1.620929,-0.988966,...,-0.988966,-0.831457,-0.988966,-0.988966,-0.988966,0.669203,0.612447,-0.988966,-0.739319,-0.988966
KYSE-520::PF-4708671,-1.188813,-1.025986,-1.025986,-0.930739,-0.930739,-0.930739,-0.930739,1.806736,1.806736,-1.188813,...,-1.188813,-1.025986,-1.188813,0.300292,0.452903,0.184968,0.816625,0.042717,0.945186,-1.188813
KYSE-70::KU-60019,-1.335412,-1.175207,0.179896,-1.081493,-0.431236,-0.431236,-1.081493,1.440960,1.440960,0.621227,...,-1.335412,0.395822,-1.335412,1.038820,-1.335412,0.174450,0.587422,0.470796,0.792185,-1.335412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"SNU-423::Brivanib, BMS-540215",-1.049877,0.814532,-0.891829,-0.799377,-0.363736,-0.363736,1.699472,1.688131,1.688131,-1.049877,...,-1.049877,-0.006160,-1.049877,0.515398,0.625330,0.229471,0.705072,-1.049877,0.950254,-1.049877
ML-2::CI-1040,0.207249,0.734708,1.068021,0.782779,-0.068855,-0.084288,0.606725,1.381015,1.381015,-1.598369,...,0.266215,0.110689,0.110623,0.459282,0.145399,0.474718,0.694607,0.396345,0.953487,0.476604
SKM-1::SB216763,0.313525,0.746472,1.126035,1.374330,1.599330,1.602073,-0.014463,1.249753,1.249753,0.052083,...,-0.105596,0.733894,0.592054,0.768386,0.203571,0.427739,0.602150,0.553736,0.892469,0.416841
NOMO-1::JNJ38877605,0.227541,0.372012,1.147412,0.988016,-0.347299,-0.347299,0.698405,1.322181,1.322181,-1.542755,...,0.560431,0.877801,0.726041,1.062889,0.611177,0.564771,0.704113,0.389156,1.005190,0.363186


In [ ]:
#phos
#finds a test train split then finds the learning curve
#for that split. Repeats for mutiple (N) test train splits 
N = 30
t1 = time.time()
for run in range(0, N):
    print(f'run {run} of {N}')
    #test train split
    rand_seed = 42 + run
    pairs_with_truth_vals =  y_list.index
    train_pairs, test_pairs, val_pairs = tts_nb.split(
        rand_seed, _all_cls, _all_drugs, pairs_with_truth_vals,
        train_size=_train_size)

    #rna test train selection
    x_train_rna, x_test_rna = x_all.loc[train_pairs], x_all.loc[test_pairs]
    x_val_rna = x_all.loc[val_pairs]
    y_train, y_test = y_list[train_pairs], y_list[test_pairs]
    y_val = y_list[val_pairs]
    xdrug_train, xdrug_test = x_drug.loc[train_pairs], x_drug.loc[test_pairs]
    xdrug_val = x_drug.loc[val_pairs]

    #prot test train selection
    x_train_prot, x_test_prot = x_all_prot.loc[train_pairs], x_all_prot.loc[test_pairs]
    x_val_prot = x_all_prot.loc[val_pairs]

    #one hot test train seleciton
    x_train_hot, x_test_hot = x_hot.loc[train_pairs], x_hot.loc[test_pairs]
    x_val_hot = x_hot.loc[val_pairs]
    
    #phos train test selection 
    x_train_phos = x_all_phos.loc[train_pairs] 
    x_test_phos = x_all_phos.loc[test_pairs]
    x_val_phos = x_all_phos.loc[val_pairs]
    
    #consistencey checks
    assert (x_train_hot.index == x_train_rna.index).all()
    assert (x_test_hot.index == x_test_rna.index).all()
    assert (x_val_hot.index == x_val_rna.index).all()

    assert (x_train_prot.index == x_train_rna.index).all()
    assert (x_test_prot.index == x_test_rna.index).all()
    assert (x_val_prot.index == x_val_rna.index).all()

    assert (y_train.index == x_train_rna.index).all()
    assert (y_test.index == x_test_rna.index).all()
    assert (xdrug_test.index == x_test_rna.index).all()
    
    assert (x_train_phos.index == x_train_rna.index).all()
    assert (x_test_phos.index == x_test_rna.index).all()
    assert (x_val_phos.index == x_val_rna.index).all()

    #inconsistencey checks
    assert x_train_rna.shape[1] != x_train_prot.shape[1]
    assert x_test_rna.shape[1] != x_test_prot.shape[1]
    assert x_val_rna.shape[1] != x_val_prot.shape[1]

    assert x_train_rna.shape[1] != x_train_hot.shape[1]
    assert x_test_rna.shape[1] != x_test_hot.shape[1]
    assert x_val_rna.shape[1] != x_val_hot.shape[1]

    assert x_train_prot.shape[1] != x_train_hot.shape[1]
    assert x_test_prot.shape[1] != x_test_hot.shape[1]
    
    assert x_train_phos.shape[1] != x_train_rna.shape[1]
    assert x_test_phos.shape[1] != x_test_rna.shape[1]
    assert x_val_phos.shape[1] != x_val_rna.shape[1]
    
    del x_train_rna, x_test_rna, x_val_rna
    del x_train_hot, x_test_hot, x_val_hot
    del x_train_prot, x_test_prot, x_val_prot
    
    #---------- Make sure correct dtype -----------------
    data_type = 'Phos_LTL'    
    #run the learning curve
    _input_shape = x_train_phos.shape[1]
    mse_r2, bms, bhps = lc_nb.run_lc_ucl(
        build_cnn_kt,
        [x_train_phos, xdrug_train], 
        y_train, 
        [x_val_phos, xdrug_val], 
        y_val, 
        [x_test_phos, xdrug_test],
        y_test, 
        lg_space, 
        num_trails=15,
        epochs=100,
        direc='UCL-del4')
    
    #save data
    mse_r2.to_csv(f'LC-metric-results/{data_type}/run{run}')
    
    bhps_df = pd.DataFrame([bhp.values for bhp in bhps])
    bhps_df.to_csv(f'Optimal-hyperparameters/{data_type}/run{run}df')
    with open(f'Optimal-hyperparameters/{data_type}/run{run}.pkl', 'wb') as f:
        pickle.dump(bhps, f)
        
    scratch_path = '/data/scratch/wpw035/In-use/DRP/rna-phos-intersect-models'
    model_path = f'{scratch_path}/NN-models/{data_type}/run{run}model_train_size_'    
    for train_size, model in zip(lg_space, bms):
        model.save(model_path + str(train_size)) 
        
    np.savetxt(f'train_test_inds/{data_type}/train_inds{run}', y_train.index, fmt='%s')
    np.savetxt(f'train_test_inds/{data_type}/test_inds{run}', y_test.index, fmt='%s')
    np.savetxt(f'train_test_inds/{data_type}/val_inds{run}', y_val.index, fmt='%s')
    
delt = time.time() - t1

Trial 15 Complete [00h 00m 15s]
val_loss: 1.8876252174377441

Best val_loss So Far: 1.7662051916122437
Total elapsed time: 00h 05m 18s
INFO:tensorflow:Oracle triggered exit
0.9814814814814815

Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
filts             |32                |?                 
units             |224               |?                 
units_hid         |224               |?                 
lr                |0.0001            |?                 

Epoch 1/100
105/105 [==============================] - 2s 11ms/step - loss: 6.5279 - mae: 2.0133 - val_loss: 7.4421 - val_mae: 2.2355
Epoch 2/100
105/105 [==============================] - 1s 7ms/step - loss: 6.2638 - mae: 1.9710 - val_loss: 7.3285 - val_mae: 2.1947
Epoch 3/100
105/105 [==============================] - 1s 7ms/step - loss: 6.1050 - mae: 1.9402 - val_loss: 6.9239 - val_mae: 2.0228
Epoch 4/100
105/105 [==============================] - 1s 7ms/step - loss: 5.8482 - mae: 1.8923